# Verifying depthwise convolutional

In [1]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import DepthwiseConv2D

from keras.optimizers import SGD
from keras.utils.generic_utils import CustomObjectScope

Using TensorFlow backend.


In [2]:
from sklearn.externals import joblib

In [3]:
keras.__version__, tf.__version__

('2.2.4', '1.13.1')

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

In [11]:
myinp = Input(shape=(6, 6, 3), dtype='float32')

x = DepthwiseConv2D(kernel_size=3,
                        strides=1, activation=None,
                        use_bias=False, padding='same',
                        name='depthwise')(myinp)
model = Model(inputs=myinp, outputs=x, name='depth')

In [12]:
model.get_weights()[0].shape, model.get_weights()[0]

((3, 3, 3, 1), array([[[[ 0.03363803],
          [ 0.13747531],
          [-0.20356108]],
 
         [[ 0.20798159],
          [ 0.1954819 ],
          [-0.07544911]],
 
         [[-0.12696448],
          [-0.27921936],
          [ 0.03870621]]],
 
 
        [[[-0.35631353],
          [ 0.01018035],
          [ 0.18393874]],
 
         [[ 0.04119885],
          [ 0.20271146],
          [-0.2119535 ]],
 
         [[-0.13997346],
          [ 0.38774323],
          [ 0.02408597]]],
 
 
        [[[ 0.39047676],
          [ 0.06395027],
          [-0.01058206]],
 
         [[-0.19540556],
          [-0.3626789 ],
          [ 0.04535034]],
 
         [[-0.19617333],
          [-0.31952235],
          [-0.14637387]]]], dtype=float32))

In [13]:
sample_inp = np.random.uniform(low=5, high=10, size=(6, 6, 3))

In [15]:
sample_inp.shape, sample_inp

((6, 6, 3), array([[[8.34473632, 9.20691313, 6.75594719],
         [6.17648446, 7.70707704, 6.83262872],
         [8.79980456, 5.42970952, 7.6073047 ],
         [5.89665535, 6.2533346 , 5.40878745],
         [9.11861739, 7.75536901, 6.30823125],
         [6.67162554, 6.63894733, 6.12837308]],
 
        [[8.13961244, 6.67552489, 7.61575606],
         [5.32336635, 6.20497389, 7.71360592],
         [8.6448741 , 7.66535925, 8.88136393],
         [6.6080001 , 7.7758087 , 9.05454875],
         [8.84482539, 7.15571847, 9.04224821],
         [8.56536526, 9.95329546, 9.21209466]],
 
        [[6.88223449, 7.06351402, 5.10538336],
         [6.69582129, 9.30273212, 6.60366747],
         [8.41161779, 5.82835107, 5.81698364],
         [9.67535904, 6.9537424 , 9.60055689],
         [9.48699562, 9.84318561, 8.12006742],
         [5.29025523, 8.85887265, 6.39561255]],
 
        [[5.02009958, 7.7011881 , 8.70539324],
         [8.3544749 , 6.61128592, 9.69160084],
         [9.3364419 , 7.01028702, 9.1828

In [16]:
with sess.as_default():
    outp = sess.run(model.output, feed_dict={model.input: sample_inp.reshape(1, 6, 6, 3), K.learning_phase(): 0})

In [17]:
outp.shape, outp

((1, 6, 6, 3), array([[[[-3.1555784 ,  0.45101437, -2.0510693 ],
          [-3.508397  , -0.51138926, -1.0530652 ],
          [-3.5705173 , -1.2639859 , -1.2295362 ],
          [-3.819655  , -0.2863409 , -0.6020985 ],
          [-3.487586  , -1.0682931 , -1.2287151 ],
          [-1.1942444 , -1.7274969 ,  0.18348402]],
 
         [[-2.1167984 , -2.1272569 , -2.4087348 ],
          [-3.7141404 ,  0.7697273 , -2.2224922 ],
          [-2.1034384 ,  1.2661488 , -3.2123346 ],
          [-4.1485944 , -1.0620904 , -2.5949295 ],
          [-1.0548661 , -0.04480943, -2.0384307 ],
          [ 1.5663657 ,  1.8710088 , -1.8316904 ]],
 
         [[-2.2565668 , -0.29418942, -2.2228901 ],
          [-4.5742455 ,  0.06272238, -3.105716  ],
          [-2.2649312 , -0.31143308, -2.6437595 ],
          [-2.951349  ,  1.0537436 , -3.8541555 ],
          [-3.301642  , -0.29339084, -3.0111194 ],
          [ 0.852603  ,  1.9314891 , -2.07431   ]],
 
         [[-3.7522814 , -2.5349808 , -2.486169  ],
        

In [19]:
BAS_PATH = '/home/coep/pkm/nn4small/mobNet_8137/'
np.save(BAS_PATH + 'depth_out.npy', outp)
np.save(BAS_PATH + 'depth_input.npy', sample_inp.reshape(1, 6, 6 ,3))
np.save(BAS_PATH + 'filter_weights.npy', model.get_weights()[0])

## Verifying Depthwise from mob_net_ep_5

In [6]:
from keras.layers import DepthwiseConv2D, ReLU
relu6 = ReLU(6.)
# (np.random.randn(216,128), np.random.randn(216, 128))
def distill_mse(y_true, y_pred):
    return 0.5*K.mean(K.sum(K.square(y_true - K.reshape(y_pred, (-1, 128))), axis=1))
with CustomObjectScope({'relu6': keras.layers.ReLU(6.),'DepthwiseConv2D': keras.layers.DepthwiseConv2D}):
    mm = load_model('/home/coep/pkm/nn4small/mobNet_8137/mob_net_ep_5.h5', custom_objects={ 'distill_mse': distill_mse})

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [10]:
mm.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 16)   64          Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (Activation)         (None, 48, 48, 16)   0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
mobl0_conv

In [13]:
mm.get_layer("mobl1_conv_depthwise").output

<tf.Tensor 'mobl1_conv_depthwise/depthwise:0' shape=(?, 24, 24, 48) dtype=float32>

In [ ]:
with sess.as_default():
    sess.run(mm.get_layer("mobl1_conv_depthwise").output, )